In [13]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#allows to print the dataframe nicely
from IPython.core import display as ICD
!pip install plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# import additional packages to insepct data and clean them
import pandas as pd
import os 
import random 
from zipfile import ZipFile
import datetime

In [38]:
# import helping functions from the implementation file
#from proj1_helpers import load_csv_data
from proj1_helpers import *
import implementations
from additional_implementations import *
import seaborn as sns

In [4]:
# import zipped files from the github repository
data_folder='./data/'
zip_file = ZipFile(data_folder+'all.zip')
# zip file creates a list of files with certain properties
zip_file.infolist()

[<ZipInfo filename='train.csv' compress_type=deflate file_size=48103379 compress_size=17670847>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=109335584 compress_size=40089137>,
 <ZipInfo filename='sample-submission.csv' compress_type=deflate file_size=5682394 compress_size=1263762>]

In [5]:
# now we want to access the 'filename' property in the zipfile variable
# and we create a dictionary of dataframe
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df_train=dfs['train.csv']
df_test=dfs['test.csv']
df_sample_submission=dfs['sample-submission.csv']

In [6]:
df_train_pca=df_train.drop(columns=['Prediction', 'Id'])

In [7]:
counts={}
for column_name in df_train_pca.columns.values:
    
    check=df_train_pca[column_name]==-999.0
    counts[column_name]=np.sum(check.astype('int'))
counts

{'DER_mass_MMC': 38114,
 'DER_mass_transverse_met_lep': 0,
 'DER_mass_vis': 0,
 'DER_pt_h': 0,
 'DER_deltaeta_jet_jet': 177457,
 'DER_mass_jet_jet': 177457,
 'DER_prodeta_jet_jet': 177457,
 'DER_deltar_tau_lep': 0,
 'DER_pt_tot': 0,
 'DER_sum_pt': 0,
 'DER_pt_ratio_lep_tau': 0,
 'DER_met_phi_centrality': 0,
 'DER_lep_eta_centrality': 177457,
 'PRI_tau_pt': 0,
 'PRI_tau_eta': 0,
 'PRI_tau_phi': 0,
 'PRI_lep_pt': 0,
 'PRI_lep_eta': 0,
 'PRI_lep_phi': 0,
 'PRI_met': 0,
 'PRI_met_phi': 0,
 'PRI_met_sumet': 0,
 'PRI_jet_num': 0,
 'PRI_jet_leading_pt': 99913,
 'PRI_jet_leading_eta': 99913,
 'PRI_jet_leading_phi': 99913,
 'PRI_jet_subleading_pt': 177457,
 'PRI_jet_subleading_eta': 177457,
 'PRI_jet_subleading_phi': 177457,
 'PRI_jet_all_pt': 0}

In [25]:
#df_train_pca=df_train_pca.drop(columns=['PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'DER_lep_eta_centrality', 'DER_prodeta_jet_jet', 'DER_mass_jet_jet', 'DER_deltaeta_jet_jet','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi'])

In [26]:
#median=np.median(df_train_pca['DER_mass_MMC'])
#df_train_pca['DER_mass_MMC'][df_train_pca['DER_mass_MMC']==-999.0]=median

In [8]:
df_train_pca[df_train_pca==-999.0]=0

### PCA

In [40]:
features = df_train_pca.columns.values
df_train_pca_std=pd.DataFrame()

df_train_pca_std, mean_train, std_train=standardize_personal(df_train_pca)
df_train_pca_std.head()
print(len(features))

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,0.513222,0.068332,0.407680,-0.469966,0.147669,0.061716,1.476969,0.882478,1.033099,0.339894,...,-0.147267,0.386847,1.044402,0.263213,1.556350,0.321296,0.935741,1.136122,-2.528290,0.412510
1,0.840891,0.552505,0.540136,-0.153167,-0.484693,-0.395606,0.121248,1.404888,-0.756027,-0.287584,...,-1.051683,-0.357719,0.021305,-0.074869,0.525758,0.829460,-0.534065,0.003141,0.000469,-0.273820
2,-1.506293,3.195156,1.096560,-0.349710,-0.484693,-0.395606,0.121248,0.989770,-0.430168,0.340361,...,-1.200672,0.400135,0.021305,-0.106352,1.486197,-1.438062,-0.534065,0.003141,0.000469,-0.293970
3,0.592488,0.910379,-0.005853,-0.903016,-0.484693,-0.395606,0.121248,1.196690,-0.830735,-0.712705,...,0.038692,-0.978149,-1.001792,-0.811735,0.001422,0.005295,-0.534065,0.003141,0.000469,-0.745439
4,1.058594,-0.914556,1.313369,-0.651804,-0.484693,-0.395606,0.121248,1.938794,-0.112795,-0.868143,...,-0.475042,-1.238475,-1.001792,-0.811735,0.001422,0.005295,-0.534065,0.003141,0.000469,-0.745439


In [10]:
cov_df_train_pca_std=np.cov(df_train_pca_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_df_train_pca_std)

In [11]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

In [31]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,21)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,21)], 
        y=cum_var_exp,
        name='Cumulative Explained Variance',
        showlegend=False)

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent', titlefont=dict(size=20)),
        xaxis=XAxis(title='Principal Components',titlefont=dict(size=20)),
        font=dict(
            family='sans-serif',
            size=17,
            color='#000'
        ))

fig = Figure(data=data, layout=layout)
py.iplot(fig)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:558: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:531: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis




In [32]:
number_features=30
#decide how many principal components i get
number_pa=25
#define matrix to be filled in
matrix_w=np.ones((number_features, number_pa))
for i in range(number_pa):
    matrix_w[:,i] = eig_pairs[i][1]

In [33]:
df_train_pca_std_transf = df_train_pca_std.dot(matrix_w)

In [34]:
y_train_pca=np.array(df_train['Prediction'])
y_train_pca[np.where(y_train_pca=='b')] = -1.
y_train_pca[np.where(y_train_pca=='s')] = 1.
y_train_pca=y_train_pca.astype('float')

In [36]:
lambdas=np.logspace(-10,-1,2)
degrees=np.linspace(5,15,3).astype('int')
mses= np.zeros((len(degrees), len(lambdas)))

for ind_degree,degree in enumerate(degrees):
    tx_train_pca_std_transf_polynomial=polynomial_features_simple(df_train_pca_std_transf, degree)
    for ind_lambda,lambda_ in enumerate(lambdas):
        w,loss=implementations.ridge_regression(y_train_pca,tx_train_pca_std_transf_polynomial,lambda_)
        mses[ind_degree, ind_lambda]=loss
        
mse_final, degree_final,lambda_final=get_best_parameters(degrees, lambdas, mses)
tx_train_pca_std_transf_polynomial=polynomial_features_simple(df_train_pca_std_transf, degree_final)
w_final,loss=implementations.ridge_regression(y_train_pca,tx_train_pca_std_transf_polynomial,lambda_final)

print('MSE: ',mse_final,'   Degree: ',degree_final,'   Lambda: ',lambda_final)
print('')
print('Ws: ',w_final)

MSE:  0.3128958907234863    Degree:  10    Lambda:  1e-10

Ws:  [-1.99480760e-02 -1.10954791e-01 -3.77521487e-02 -3.48138051e-03
  1.79570536e-03  6.86045146e-04  9.97033175e-05  7.50215942e-06
  3.09021719e-07  6.61508003e-09  5.76003885e-11 -2.76706260e-02
 -4.22440081e-02  5.26730524e-03  2.37612856e-03 -4.44076563e-05
 -4.37211621e-05 -1.60957512e-06  2.20649625e-07  1.74096444e-08
  3.45269098e-10 -1.34271244e-01 -1.70884104e-02  6.09594495e-03
  7.50112147e-04 -1.52354612e-04 -1.44027500e-05  1.31776765e-06
  1.08042697e-07 -3.12525610e-09 -2.37893636e-10 -2.51912359e-01
  2.27626493e-02  1.46600455e-02 -1.20894938e-03 -2.54642334e-04
  1.69299510e-05  1.54947669e-06 -9.06523973e-08 -2.94950778e-09
  1.61986396e-10  4.30649183e-03 -3.68710924e-02  4.69915063e-03
 -1.54123551e-03 -1.25238717e-03 -3.97855388e-05  1.33034440e-04
  9.28895605e-05 -4.29880886e-06 -4.70676761e-06 -9.75255467e-03
  5.39240367e-02 -6.44825934e-03 -5.24377038e-02  5.09806206e-03
  1.72126541e-02 -7.830852

In [35]:
y_pred=predict_labels(w_final,tx_train_pca_std_transf_polynomial)
print('Performance: ',list(y_pred*y_train_pca).count(1.)/len(tx_train_pca_std_transf_polynomial))
print('Degree: ',degree_final)
print('Lambda: ',lambda_final)

Performance:  0.77598
Degree:  13
Lambda:  1e-06


In [ ]:
df_test_pca=df_test.drop(columns=['Prediction', 'Id'])

In [ ]:
df_test_pca=df_test_pca.drop(columns=['PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'DER_lep_eta_centrality', 'DER_prodeta_jet_jet', 'DER_mass_jet_jet', 'DER_deltaeta_jet_jet','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi'])

In [ ]:
df_test_pca['DER_mass_MMC'][df_test_pca['DER_mass_MMC']==-999.0]=median

In [ ]:
features = df_test_pca.columns.values
df_test_pca_std=pd.DataFrame()

# def standardize_test(x, train_set):
#     x=(x-np.mean(train_set, axis=0))/np.std(train_set, axis=0)
#     return x

df_test_pca_std=standardize_test(df_test_pca, mean_train, std_train)
df_test_pca_std.head()

In [ ]:
df_test_pca_std_transf = df_test_pca_std.dot(matrix_w)

In [ ]:
tx_test_pca_std_transf_polynomial=polynomial_features_simple(df_test_pca_std_transf, degree_final)

# Submission

In [ ]:
tx_test=tx_test_pca_std_transf_polynomial
y_pred=predict_labels(w_final,tx_test)
create_csv_submission(df_test['Id'], y_pred, 'trial_ridge_polynomial_pca')